### Se carga las librerias

In [178]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import openpyxl

In [179]:
#import openpyxl

#### Se carga el archivo generado por el main.py que hace el scrapping

In [180]:
#path = "c:/Users/mhrec/OneDrive/Capacitación/AmazonWebScrapping/"
path = "E:/Proyectos de Python/AmazonWebScrapping/"

df = pd.read_csv(path+ "amazon_products.csv")

In [181]:
df.shape

(210, 6)

In [182]:
df.head(61)

,Book Name,Author,Rating,Customers_Rated,Price,Quality_Book
0,Atomic Habits: the life-changing million-copy ...,James Clear,4.6 out of 5 stars,"82,875",₹460.00,Paperback
1,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5 out of 5 stars,"72,090",₹399.00,Board book
2,The Psychology Of Money,Morgan Housel,4.6 out of 5 stars,"53,486",₹259.00,Paperback
3,Ikigai,Francesc Miralles,4.6 out of 5 stars,"49,898",₹265.00,Hardcover
4,Colouring Books for Kids (Pack of 12 Books),Wonder House Books,4.5 out of 5 stars,"3,721",₹299.00,Paperback
...,...,...,...,...,...,...
56,THE POWER OF YOUR SUBCONSCIOUS MIND,Joseph Murphy,4.5 out of 5 stars,"76,908",₹115.00,Paperback
57,R D Sharma Mathematics Class 10 with MCQ in Ma...,R.D. Sharma,4.5 out of 5 stars,"2,928",₹530.00,Paperback
58,Grandparents' Bag of Stories [Paperback] Sudha...,Sudha Murty,4.7 out of 5 stars,Sudha Murty,₹151.00,Paperback
59,Think and Grow Rich THE 21st CENTURY EDITION,Napoleon Hill,4.5 out of 5 stars,"4,747",₹99.00,Paperback


In [183]:
df.to_excel('amazon_products_test_2.xlsx', index=False)

## Let's do some preprocessing on the Amazon ratings, customers_rated, and price column.

In [184]:
df['Rating'] = df['Rating'].apply(lambda x: x.split()[0]) # remover el texto sobrante

In [185]:
df["Price"] = df["Price"].str.replace('₹', '') # remover el simbolo de yuanes

In [186]:
df["Price"] = df["Price"].str.replace(',', '') # remover la coma de miles y millones (de existir)

In [187]:
df['Price'] = df['Price'].apply(lambda x: x.split('.')[0]) # eliminar los centavos

In [188]:
df['Price'] = df['Price'].astype(int) #precio entero

In [189]:
df["Customers_Rated"] = df["Customers_Rated"].str.replace(',', '')

In [190]:
df['Customers_Rated'] = pd.to_numeric(df['Customers_Rated'], errors='ignore')

In [191]:
df['Customers_Rated'] = pd.to_numeric(df['Customers_Rated'], errors='coerce') 
# se elimina las filas no numéricas y se coloca na // solución temporal hasta arreglar el scrapping

In [192]:
df = df.dropna(subset=['Customers_Rated'])
# se elimina las filas no numéricas y se coloca na // solución temporal hasta arreglar el scrapping

In [193]:
df['Customers_Rated'] = df['Customers_Rated'].astype(int) #precio entero

In [194]:
df.head()

,Book Name,Author,Rating,Customers_Rated,Price,Quality_Book
0,Atomic Habits: the life-changing million-copy ...,James Clear,4.6,82875,460,Paperback
1,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5,72090,399,Board book
2,The Psychology Of Money,Morgan Housel,4.6,53486,259,Paperback
3,Ikigai,Francesc Miralles,4.6,49898,265,Hardcover
4,Colouring Books for Kids (Pack of 12 Books),Wonder House Books,4.5,3721,299,Paperback


In [195]:
df.dtypes

Book Name          object
Author             object
Rating             object
Customers_Rated     int32
Price               int32
Quality_Book       object
dtype: object

In [196]:
df['Rating'] = df['Rating'].astype(float)

In [197]:
df.head()

,Book Name,Author,Rating,Customers_Rated,Price,Quality_Book
0,Atomic Habits: the life-changing million-copy ...,James Clear,4.6,82875,460,Paperback
1,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5,72090,399,Board book
2,The Psychology Of Money,Morgan Housel,4.6,53486,259,Paperback
3,Ikigai,Francesc Miralles,4.6,49898,265,Hardcover
4,Colouring Books for Kids (Pack of 12 Books),Wonder House Books,4.5,3721,299,Paperback


In [198]:
df.dtypes

Book Name           object
Author              object
Rating             float64
Customers_Rated      int32
Price                int32
Quality_Book        object
dtype: object

Replace the zero values in the DataFrame to NaN.

In [199]:
df.replace(str(0), np.nan, inplace=True)
df.replace(0, np.nan, inplace=True)

### Counting the Number of NaNs in the DataFrame

In [200]:
count_nan = len(df) - df.count()

In [201]:
count_nan

Book Name          0
Author             3
Rating             0
Customers_Rated    0
Price              0
Quality_Book       1
dtype: int64

In [202]:
df = df.dropna()

### Authors' Highest Priced Book on Amazon
Let's find out which authors had the highest-priced book. You will visualize the results for such the top 20 authors.

In [203]:
data = df.sort_values(["Price"], axis=0, ascending=False)[:21]

In [204]:
data

,Book Name,Author,Rating,Customers_Rated,Price,Quality_Book
156,Harry Potter Box Set: The Complete Collection ...,J.K. Rowling,4.7,63097,2767,Paperback
119,Errorless 46 Previous Years IIT JEE Advanced (...,Disha Experts,4.5,108,1420,Paperback
140,Harry Potter 1–3 Box Set: A Magical Adventure ...,J.K. Rowling,4.6,3449,1103,Paperback
111,Oswaal CAT 25 Years' Chapter-wise and Topic-wi...,Oswaal Editorial Board,4.3,289,865,Paperback
9,Indian Polity for UPSC (English| 7th Edition) ...,M Laxmikanth,4.5,225,744,Paperback
123,The Original Rider Waite Tarot Pack [Cards] Wa...,A.E. Waite,4.5,8094,717,Cards
163,Trading in the Zone,Mark Douglas,4.5,7364,714,Hardcover
175,R D Sharma Mathematics for Class 11 (Set of 2 ...,R.D. Sharma,4.4,205,698,Paperback
180,Indian Art and Culture,Nitin Singhania,4.6,1865,684,Paperback
27,MTG Objective NCERT at your FINGERTIPS Biology...,MTG Editorial Board,4.4,619,649,Paperback


### Librerias para Loading BokehJS ...

#### install Bokeh

In [205]:
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge
import math
from bokeh.io import curdoc
curdoc().clear()
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.models import Legend
output_notebook()

Loading BokehJS ...

In [206]:
p = figure(x_range=data.iloc[:,1], width=800, height=550, title="Authors Highest Priced Book", toolbar_location=None, tools="")

p.vbar(x=data.iloc[:,1], top=data.iloc[:,4], width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = math.pi/2

# check that plot_width and plot_height fueron reemplazados por width y con height

In [207]:
show(p)

ERROR:bokeh.core.validation.check:E-1019 (DUPLICATE_FACTORS): FactorRange must specify a unique list of categorical factors for an axis: duplicate factors found: 'J.K. Rowling', 'Oswaal Editorial Board', 'R.D. Sharma', 'MTG Editorial Board'


In [208]:
data2 = df.sort_values(["Price"], axis=0, ascending=False)[:5]

In [209]:
data2

,Book Name,Author,Rating,Customers_Rated,Price,Quality_Book
156,Harry Potter Box Set: The Complete Collection ...,J.K. Rowling,4.7,63097,2767,Paperback
119,Errorless 46 Previous Years IIT JEE Advanced (...,Disha Experts,4.5,108,1420,Paperback
140,Harry Potter 1–3 Box Set: A Magical Adventure ...,J.K. Rowling,4.6,3449,1103,Paperback
111,Oswaal CAT 25 Years' Chapter-wise and Topic-wi...,Oswaal Editorial Board,4.3,289,865,Paperback
9,Indian Polity for UPSC (English| 7th Edition) ...,M Laxmikanth,4.5,225,744,Paperback


In [210]:
p = figure(x_range=data2.iloc[:,1], width=800, height=550, title="Authors Highest Priced Book", toolbar_location=None, tools="")

p.vbar(x=data2.iloc[:,1], top=data2.iloc[:,4], width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = math.pi/2

# check that plot_width and plot_height fueron reemplazados por width y con height

In [211]:
show(p)

ERROR:bokeh.core.validation.check:E-1019 (DUPLICATE_FACTORS): FactorRange must specify a unique list of categorical factors for an axis: duplicate factors found: 'J.K. Rowling'


### Remove Duplicates in One Column and Keep Row with Max

In [212]:
data3 = data.sort_values('Price', ascending=False).drop_duplicates('Author').sort_index()

In [213]:
data3

,Book Name,Author,Rating,Customers_Rated,Price,Quality_Book
9,Indian Polity for UPSC (English| 7th Edition) ...,M Laxmikanth,4.5,225,744,Paperback
27,MTG Objective NCERT at your FINGERTIPS Biology...,MTG Editorial Board,4.4,619,649,Paperback
86,Sapiens: THE MULTI-MILLION COPY BESTSELLER,Yuval Noah Harari,4.7,115347,477,Paperback
111,Oswaal CAT 25 Years' Chapter-wise and Topic-wi...,Oswaal Editorial Board,4.3,289,865,Paperback
119,Errorless 46 Previous Years IIT JEE Advanced (...,Disha Experts,4.5,108,1420,Paperback
123,The Original Rider Waite Tarot Pack [Cards] Wa...,A.E. Waite,4.5,8094,717,Cards
124,The Kalki Trilogy Set of 3 Books Avatar of Vis...,Kevin Missal,4.8,15,599,Paperback
141,Concept of Physics by H.C Verma Part - I - Ses...,H.C. Verma,4.5,13474,544,Paperback
156,Harry Potter Box Set: The Complete Collection ...,J.K. Rowling,4.7,63097,2767,Paperback
163,Trading in the Zone,Mark Douglas,4.5,7364,714,Hardcover


In [214]:
p = figure(x_range=data3.iloc[:,1], width=800, height=550, title="Authors Highest Priced Book", toolbar_location=None, tools="")

p.vbar(x=data3.iloc[:,1], top=data3.iloc[:,4], width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = math.pi/2

# check that for bokeh to work, it was necessary to remove duplicates from Autor name

In [215]:
show(p)

### Top Rated Books by Authors
Let's find out which authors have the top-rated books and which books of those authors are top rated. However, while finding this out, you would filter out those authors in which less than 1000 customers rated.

In [216]:
data3 = df[df['Customers_Rated'] > 1000]

In [217]:
data3

,Book Name,Author,Rating,Customers_Rated,Price,Quality_Book
0,Atomic Habits: the life-changing million-copy ...,James Clear,4.6,82875,460,Paperback
1,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5,72090,399,Board book
2,The Psychology Of Money,Morgan Housel,4.6,53486,259,Paperback
3,Ikigai,Francesc Miralles,4.6,49898,265,Hardcover
4,Colouring Books for Kids (Pack of 12 Books),Wonder House Books,4.5,3721,299,Paperback
...,...,...,...,...,...,...
205,Harry Potter and the Philosopher's Stone,J.K. Rowling,4.7,101196,342,Paperback
206,Why Has Nobody Told Me This Before?: The No 1 ...,Dr Julie Smith,4.5,8599,388,Paperback
207,Godan (Hindi),Premchand,4.5,6565,188,Paperback
208,"Red, White and Royal Blue",Casey McQuiston,4.4,33535,410,Paperback


In [218]:
data3 = data3.sort_values(['Rating'],axis=0, ascending=False)[:15]


In [219]:
data3


,Book Name,Author,Rating,Customers_Rated,Price,Quality_Book
11,Shrimad Bhagwat Geeta Yatharoop,A.C. Bhaktivendanta Swami Prabhupada,4.8,17009,227,Hardcover
86,Sapiens: THE MULTI-MILLION COPY BESTSELLER,Yuval Noah Harari,4.7,115347,477,Paperback
20,KARMA: A YOGI'S GUIDE TO CRAFTING YOUR DESTINY,Sadhguru,4.7,14399,192,Paperback
156,Harry Potter Box Set: The Complete Collection ...,J.K. Rowling,4.7,63097,2767,Paperback
164,"The Boy, The Mole, The Fox and The Horse [Hard...",Charlie Mackesy,4.7,68949,628,Hardcover
39,Bhagavad Gita [Deluxe Hardbound Edition] Sacre...,Anonymous,4.7,1610,219,Hardcover
201,Sunderkand,Goswami Tulsidas,4.7,3798,109,Paperback
205,Harry Potter and the Philosopher's Stone,J.K. Rowling,4.7,101196,342,Paperback
182,"Hidden Hindu, The ( Book 2)",Akshat Gupta,4.6,1112,180,Paperback
180,Indian Art and Culture,Nitin Singhania,4.6,1865,684,Paperback


In [220]:
p = figure(x_range=data3.iloc[:,0], width=800, height=600, title="Top Rated Books with more than 1000 Customers Rating", toolbar_location=None, tools="")

p.vbar(x=data3.iloc[:,0], top=data3.iloc[:,2], width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 1000
p.xaxis.major_label_orientation = math.pi/2


In [221]:
show(p)
### hay que arreglar este grafico

From the above output, you can observe that the top three rated books with more than 1000 customer ratings are Inner Engineering: A Yogi’s Guide to Joy, Bhagavad-Gita (Hindi), and The Alchemist.

In [222]:
data4 = data3.sort_values('Customers_Rated', ascending=False).drop_duplicates('Author').sort_index()

In [223]:
p = figure(x_range=data4.iloc[:,1], width=800, height=600, title="Top Rated Books with more than 1000 Customers Rating", toolbar_location=None, tools="")

p.vbar(x=data4.iloc[:,1], top=data4.iloc[:,2], width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = math.pi/2



In [224]:
show(p)


### Most Customer Rated Authors and Books
While you have already seen the top-rated books and top-rated authors, it would still be more convincing and credible to conclude the best author and the book based on the number of customers who rated for that book.

So, let's quickly find that out

In [225]:
data5 = df.sort_values(["Customers_Rated"], axis=0, ascending=False)[:20]


In [226]:
data5


,Book Name,Author,Rating,Customers_Rated,Price,Quality_Book
17,"THE SILENT PATIENT [Paperback] Michaelides, Alex",Alex Michaelides,4.5,264042,194,Paperback
82,"VERITY [Paperback] Hoover, Colleen",Colleen Hoover,4.3,245922,234,Paperback
86,Sapiens: THE MULTI-MILLION COPY BESTSELLER,Yuval Noah Harari,4.7,115347,477,Paperback
205,Harry Potter and the Philosopher's Stone,J.K. Rowling,4.7,101196,342,Paperback
0,Atomic Habits: the life-changing million-copy ...,James Clear,4.6,82875,460,Paperback
56,THE POWER OF YOUR SUBCONSCIOUS MIND,Joseph Murphy,4.5,76908,115,Paperback
7,The Power of Your Subconscious Mind,Joseph Murphy,4.5,76908,115,Paperback
47,The Power of Your Subconscious Mind,Joseph Murphy,4.5,76908,105,Paperback
122,The Power of your Subconscious Mind,Joseph Murphy,4.5,76908,145,Paperback
1,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5,72090,399,Board book


In [227]:
from bokeh.transform import factor_cmap
from bokeh.models import Legend
from bokeh.palettes import Dark2_5 as palette
import itertools
from bokeh.palettes import d3
#colors has a list of colors which can be used in plots
colors = itertools.cycle(palette)

palette = d3['Category20'][20]


In [228]:
index_cmap = factor_cmap('Author', palette=palette,
                         factors=data5["Author"])


In [229]:
p = figure(width=700, height=700, title = "Top Authors: Rating vs. Customers Rated")
p.scatter('Rating','Customers_Rated',source=data5,fill_alpha=0.6, fill_color=index_cmap,size=20,legend_field='Author')
p.xaxis.axis_label = 'RATING'
p.yaxis.axis_label = 'CUSTOMERS RATED'
p.legend.location = 'top_left'


In [230]:
show(p)


The above graph is a scatter plot of Authors who bagged customer rating vs. actual rating. The following conclusions can be made after looking at the above plot.

Hands down Paulo Coelho's book The Alchemist is the best selling book since the rating, and the number of customers rated are both in sync.
The author Amish Tripathi's book Ram - Scion of Ikshvaku (Ram Chandra) has a rating of 4.2 with a 5766 customer rating. However, the author George S. Clason's book The Richest Man in Babylon has almost similar customers rating, but the overall rating is 4.5. Hence, it can be concluded that more customers gave a high rating to The Richest Man in Babylon